In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import pyodbc
import PIL

# SQL data source extractions from database

## Getting the Unit installation and Removal 

Since I don't want to share the information of the SQL database, I will simply explain how the query was made to get the time between two unit installation.

The database where the information of the units is located is a transactional database. In other words, each time an action is created, a new data entry is made. In our case, we wanted to get the moment a unit was installed and remove. To acquire this information I have to do a self join and get the minimum value between two installations. 

First problematic: The self join query cannot be done on the unit serial number since there is no restraint on the user data entry. For example, the same unit could have the following serial number FV-0001 or FV0001 or 0001.

Since it is not possible to self join the table on the serial number, I have to self join the table on main_unit number and unit position. 

This will help us get the lifespan, moment of installation, moments of removal and unit functionning time.


## Getting The Active_lifespan

The second query will help get the real running moment of the unit. Since the unit is installed and doesn't run all the time, it will be interesting to get a clear view on how long each unit ran before the being removed. The query is a simple select.

## Getting the incident

During a unit lifespan, it is possible that an event occur which can be clear indicator of the malfunction of a unit. Luckily, there is an existing SQL database available to extract this information. The query is a simple select.

In [ ]:
#Python query example with a simple SQL select

mySql_query= """Select * from Table
     """
connection = pyodbc.connect(driver='{SQL Server}', 
                                server='Serveur1', 
                                database='Database',               
                                trusted_connection='yes')
Data_Unit=pd.read_sql(mySql_query, connection)


# END OF SQL 

# SNAPSHOT FUNCTIONS

Explanation of the code next steps:

Right now, the data shows when the unit was installed and when the unit was removed. Since we are trying to predict when to remove the units, we need to modify the current dataset to represent reality. The course of action decided will be to create a time series by partitioning the data. 

For example, if a unit was installed from the 2019-01-01 to the 2020-01-01, it means it was installed 365 days. We can partition the data into four (each 100 days) to get the desired snapshots. Those snapshot will be used to represent the moment we run the model when it is in production.

The next steps will be to add features between the unit installation date and the snapshot date. This step will create the cumulative time series that we will be using to train the model. 

In other words, the model will be trained based on time series that represent a cumulation of events of features.


In [ ]:
### This funciton is responsible for created the snapshot on the data. You can see that data is partitionned 
### every 20 days.
### Since it is still unclear if the business wants to predict the unit failure 60-90-120 days before failure

def TimeLine2(data):
    
   #BUG possible a revoir 
    Columns=['Main_Unit_Number',
'Serial_Number_Installed',
'Unit_Position',
'Serial_Number_Removed',
'Installation_date',
'Remove_date',
'Main_Unit_Type',
'Unit_Active_Time',
'Unit_Lifespan',
'Daily_Active_Time']
    position=0    
    Unit_life=pd.DataFrame(columns=Columns)
    Snapshot=[]
    
    for row in range(data.shape[0]):
        
        #For each rows, installation Date and removing Date are initialized
        insta=data.at[row,'Installation_date']
        remove=data.at[row,'Remove_date']

        while insta<remove:
            
            insta=insta+ datetime.timedelta(days=20)
            
            Unit_life=Unit_life.append(data.iloc[row:row+1])

            #Snapshot=Snapshot.append(insta)
            if insta<remove:
                Snapshot.append(insta)
            else:
                Snapshot.append(data.iloc[row,8])

    Unit_life['Snapshot_Date']=Snapshot
    return(Unit_life)

# dataframe creations functions

In [ ]:
## The count weather function calculate all the weather event that the unit experienced between the installation
## and the Snapshot moment.


def count_weather(data,Temperatures):
    

    Columns=[
    'date',
    '-15 and under',
    '-10 to -14',
    '-5 to -9',
    '-0 to -4',
    '15 and over',
    '10 to 14',
    '5 to 9',
    '1 to 4',
    'Snow',
    'Rain',
    'Ice Storm'
             ]
    newcolumns=pd.DataFrame(columns=Columns)
    
    data_with_weather=data.join(newcolumns)
    
    #Weather Date is removed
    data_with_weather=data_with_weather.drop(['date'], axis=1)
    Columns.remove('date')
    
    for row in range(data.shape[0]):
        
        #For each unit(rows), installation Date and Snapshot_Date are initialized
        insta=data.at[row, 'Installation_date'] #[row]['Installation_date']
        snap=data.at[row, 'Snapshot_Date']
        
        for col in Columns:

            #Days for each range of temperature are summed between considered Dates for each unit
            data_with_weather.at[row,col] = Temperatures[col][(Temperatures['date'] > insta) & (Temperatures['date'] <= snap)].sum()
    
    return(data_with_weather)

In [ ]:
### Main Unit Classification function
### Based on data profiling (see other document) and engineering review, the main unit can be divided
### into 2 categories.

def Type_Classification(x):
    if x=='H':
        return 1
    else:
        return 0

In [ ]:
## Since the position of the unit is not clean, it is really important to standardized it numerically. 
## Instead of having 1R, 01R 
## or 01-R. The posiontions will be standardized 1,2,3 nad 4.

def PositionFunc(position):
    for x in position:
        if x.isdigit():
            if int(x)>0:
                value=x
                return value
    return 'none'

In [ ]:
Data_Unit['Installation_date']= pd.to_datetime(Data_Unit['Installation_date']) 
Data_Unit['Remove_date']= pd.to_datetime(Data_Unit['Remove_date'])
  
Malfunction_data['Mal_Date']=pd.to_datetime(Malfunction_data['Mal_Date'])

In [ ]:
#clean the position entry

Data_Unit['Key_Position']=Data_Unit['EquipmentNumber'].astype('str')+Data_Unit['CleanPosition'].astype(str)

#Add Column
Data_Unit['Installation_month']=Data_Unit['Installation_date'].apply(lambda x:x.month)
Data_Unit['Remove_month']=Data_Unit['Remove_date'].apply(lambda x:x.month)
Data_Unit['EquipmentTypeN']=Data_Unit['EquipmentType'].apply(lambda x:EquipmentType(x))


In [ ]:
#Add Column
Data_Unit['Installation_month']=Data_Unit['Installation_date'].apply(lambda x:x.month)
Data_Unit['Remove_month']=Data_Unit['Remove_date'].apply(lambda x:x.month)
Data_Unit['EquipmentTypeN']=Data_Unit['EquipmentType'].apply(lambda x:EquipmentType(x))


In [ ]:
Data_Unit['CleanPosition']=Data_Unit['MainComponentPositionCode'].apply(PositionFunc)

In [ ]:
Data_Unit_Life=TimeLine2(Data_Unit)


In [ ]:
Data_Unit_Life['Snapshot_Date']=pd.to_datetime(Data_Unit_Life['Snapshot_Date']) 


In [ ]:
### DATA PREPARATION ####




#clean the position entry

#Should be done before


#create a position key to merge malfunction data with unit change data
Malfunction_data['Key_Position']=Malfunction_data['Main_Unit_Number'].astype(str)+Malfunction_data['position'].astype(str)


#create a super key to merge unit change with malfunction data matrix
Data_Unit_Life['super_key']=(Data_Unit_Life['Installation_date'].astype(str)+
                             Data_Unit_Life['Snapshot_Date'].astype(str)+
                             Data_Unit_Life['Key_Position'].astype(str)+
                             Data_Unit_Life['CleanPosition'].astype(str))


Malfunction_data=Data_Unit_Life.merge(right=Malfunction_data,on='Key_Position',how='left')

Malfunction_data['super_key']= (Malfunction_data['Installation_date'].astype(str)+
                          Malfunction_data['Snapshot_Date'].astype(str)+
                          Malfunction_data['Key_Position'].astype(str)+
                          Malfunction_data['position'].astype(str))

#Create sub matrix with level condition and date level condition

Mal_list_LV=[0,1,2,3]

for i in Mal_list:
    
    LV=Malfunction_data[(Malfunction_data['Installation_date']<Malfunction_data['Function_Time'])&(
        Malfunction_data['Snapshot_Date']>=Malfunction_data['Function_Time'])&(Malfunction_data['Level']==i)]

    LV=LV.groupby('super_key'
    ).count()
    LV=LV.reset_index()
    LV['L0']=LV[['Main_Unit_Number']]
    LV=LV[['super_key','L0']]

    Data_Unit_Life=Data_Unit_Life.merge(right=LV,on='super_key',how='left')






In [ ]:
datat= pd.read_csv(r'WheatherData.csv')

In [ ]:
datat['date']=pd.to_datetime(datat['date'])


In [ ]:
#reset index to enable date match
Data_Unit_Life=Data_Unit_Life.reset_index(drop=True)

#count wetaher date
Final_data=count_weather(Data_Unit_Life,datat)


#shape check fo duplicate
print(Final_data.shape)


Final_data['RUL']=[int(d.days) for d in (Final_data['Remove_date']-Final_data['Snapshot_Date'])]

In [ ]:
Final_data.to_csv('Final_data.csv')